In [1]:
import requests
from sklearn.metrics.pairwise import cosine_similarity


def contentBasedFiltering():

# user features....

# Replace these with your Firebase project details
  firebase_url = "https://ecommercestore-7733c-default-rtdb.asia-southeast1.firebasedatabase.app"
  path_to_users = "Users/Registered.json"
  path_to_orders = "Orders.json"

  # Make a GET request to fetch user data
  response_users = requests.get(f"{firebase_url}/{path_to_users}")

  user_info_list = []
  user_ids=[]
  user_vectors = []

  # Check if the request for user data was successful (status code 200)
  if response_users.status_code == 200:
      users_data = response_users.json()

      # Loop through all user data
      for firebase_user_id, user_data in users_data.items():
          # Extract user information
          user_id = user_data.get("userID")
          user_age = user_data.get("userAge", 0)
          user_gender = user_data.get("userGender", "Unknown")

          # Check if the user has placed an order
          response_orders = requests.get(f"{firebase_url}/Orders/{user_id}.json")

          if response_orders.status_code == 200:
              orders_data = response_orders.json()

              # User has placed an order
              if orders_data is not None:
                  user_category_counts = {"APPAREL": 0, "FOOTWEAR": 0, "EYEWEAR": 0, "WATCHES": 0, "JEWELLERY": 0, "BAGS": 0}

                  for order_id, order_details in orders_data.items():
                      for item in order_details["cart"]:
                          item_category = item["itemCategory"]
                          user_category_counts[item_category] += 1

                  user_vector = [user_age, 1 if user_gender == "MALE" else 0] + list(user_category_counts.values())
                  user_info_list.append([user_id, user_age, user_gender])
                  user_vectors.append(user_vector)
                  user_ids.append(user_id)
                  print(f"User ID: {user_id}, User Vector: {user_vector}")

              # User has not placed an order
              else:
                  user_vector = [user_age, 1 if user_gender == "MALE" else 0] + [ 0, 0, 0, 0, 0, 0]
                  user_info_list.append([user_id, user_age, user_gender])
                  user_vectors.append(user_vector)
                  user_ids.append(user_id)
                  print(f"User ID: {user_id}, No orders placed, User Vector: {user_vector}")

          else:
              print(f"Failed to fetch orders data for User ID {user_id}. Status code: {response_orders.status_code}")

      # Print all user vectors
      print("All User Vectors:", user_vectors)
      print("User  id's :", user_ids)


  else:
      print(f"Failed to fetch user data. Status code: {response_users.status_code}")
      users_data = {}

  # products features.......


  # Replace these with your Firebase project details
  firebase_url = "https://ecommercestore-7733c-default-rtdb.asia-southeast1.firebasedatabase.app"
  path_to_products = "Products.json"  # Assuming "products" is your collection name

  # Make a GET request to fetch data from the "products" collection in Firebase
  response = requests.get(f"{firebase_url}/{path_to_products}")

  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      product_data = response.json()

      # Assuming you have a list of products and you want to create vectors for each
      all_product_data = []
      all_product_vectors = []
      product_vectors=[]
      product_ids=[]
      for product_id, product in product_data.items():
          # Define your categories
          categories = ["APPAREL", "FOOTWEAR", "EYEWEAR", "WATCHES", "JEWELLERY", "BAGS"]

          # Create category vector
          category_vector = [1 if product["productCategory"] == category else 0 for category in categories]

          price_range = product["productPrice"]
          stock_level = product["productStock"]

          product_vector = category_vector + [price_range, stock_level]

          # Append product data and product vector to their respective lists
          all_product_data.append(product)
          product_ids.append(product["productID"])
          all_product_vectors.append({"productID": product["productID"], "productVector": product_vector})
          product_vectors.append(product_vector)
          print(f"Product ID: {product_id}, Product Vector: {product_vector}")

      # Print all product data and vectors
      print("All Product Data:", all_product_data)
      print("All Product Vectors:", all_product_vectors)
      print("Product vectors; ",product_vectors)
      print("Product ID vector..", product_ids)

  else:
      print(f"Failed to fetch data. Status code: {response.status_code}")



  # model is here..........................

# Function to recommend products based on user vector
  def recommend_products(user_vector, product_vectors, product_ids, top_n=5):
      # Calculate cosine similarity between the user vector and all product vectors
      similarities = cosine_similarity([user_vector], product_vectors)[0]

      # Get the indices of the top N similar products
      top_indices = similarities.argsort()[-top_n:][::-1]

      # Recommend products and their similarity scores
      recommendations = [(product_ids[i], similarities[i]) for i in top_indices]

      return recommendations

  # Firebase project details
  firebase_url = "https://ecommercestore-7733c-default-rtdb.asia-southeast1.firebasedatabase.app"
  path_to_users = "Users/Registered.json"
  path_to_products = "Products.json"
  path_to_recommendations = "Recommendations.json"

  # Fetch user data
  response_users = requests.get(f"{firebase_url}/{path_to_users}")

  user_info_list = []
  user_ids = []
  user_vectors = []

  # Fetch product data
  response_products = requests.get(f"{firebase_url}/{path_to_products}")

  product_data = response_products.json()

  # Assuming user data and product data are available
  if response_users.status_code == 200 and response_products.status_code == 200:
      users_data = response_users.json()

      # Loop through all user data
      for firebase_user_id, user_data in users_data.items():
          # Extract user information
          user_id = user_data.get("userID")
          user_age = user_data.get("userAge", 0)
          user_gender = user_data.get("userGender", "Unknown")

          # Check if the user has placed an order
          response_orders = requests.get(f"{firebase_url}/Orders/{user_id}.json")

          if response_orders.status_code == 200:
              orders_data = response_orders.json()

              # User has placed an order
              if orders_data is not None:
                  user_category_counts = {"APPAREL": 0, "FOOTWEAR": 0, "EYEWEAR": 0, "WATCHES": 0, "JEWELLERY": 0, "BAGS": 0}

                  for order_id, order_details in orders_data.items():
                      for item in order_details["cart"]:
                          item_category = item["itemCategory"]
                          user_category_counts[item_category] += 1

                  user_vector = [user_age, 1 if user_gender == "MALE" else 0] + list(user_category_counts.values())
                  user_info_list.append([user_id, user_age, user_gender])
                  user_vectors.append(user_vector)
                  user_ids.append(user_id)
                  print(f"User ID: {user_id}, User Vector: {user_vector}")

                  # Get product recommendations for the current user
                  recommendations = recommend_products(user_vector, product_vectors, product_ids, top_n=8)

                  # Store recommendations in Firebase under the user ID
                  recommendations_data = {user_id: [product_id for product_id, _ in recommendations]}
                  requests.patch(f"{firebase_url}/{path_to_recommendations}", json=recommendations_data)

                  # Display recommendations for the current user
                  print(f"\nRecommended Products for User ID: {user_id}")
                  for product_id, similarity in recommendations:
                      print(f"Product ID: {product_id}, Similarity: {similarity}")

              # User has not placed an order
              else:
                  user_vector = [user_age, 1 if user_gender == "MALE" else 0] + [0, 0, 0, 0, 0, 0]
                  user_info_list.append([user_id, user_age, user_gender])
                  user_vectors.append(user_vector)
                  user_ids.append(user_id)
                  print(f"User ID: {user_id}, No orders placed, User Vector: {user_vector}")

          else:
              print(f"Failed to fetch orders data for User ID {user_id}. Status code: {response_orders.status_code}")

      # Print all user vectors
      print("\nAll User Vectors:", user_vectors)
      print("User IDs:", user_ids)

  else:
      print(f"Failed to fetch user or product data. User status code: {response_users.status_code}, Product status code: {response_products.status_code}")
      users_data = {}




contentBasedFiltering()

User ID: N15GKSOGUTOM4KkXNUTXzEGBxnP2, No orders placed, User Vector: [20, 0, 0, 0, 0, 0, 0, 0]
User ID: 5OrO3eHvh8UEGMiE6bN6mAh553r2, User Vector: [32, 0, 0, 0, 0, 1, 0, 0]
User ID: pBgR6qjSR5eDKLcpUCyma5iq3se2, User Vector: [26, 0, 0, 0, 0, 3, 2, 0]
User ID: 9GGCUTU8O9aMT5vzB6TLHDGT6Hr1, User Vector: [21, 0, 0, 1, 0, 0, 0, 0]
User ID: tvkAe9bsGVZpp86PkttzDSuRwZE3, No orders placed, User Vector: [0, 1, 0, 0, 0, 0, 0, 0]
User ID: pVkzmPqUphXldhcXG7YHbe31sFB3, No orders placed, User Vector: [20, 1, 0, 0, 0, 0, 0, 0]
User ID: b7025ce3-c24d-4cf7-8664-1994a1ab0442, No orders placed, User Vector: [23, 1, 0, 0, 0, 0, 0, 0]
User ID: 974d13ad-05b9-4f3c-9b0f-d4cf4cad445b, User Vector: [23, 1, 5, 0, 2, 0, 1, 0]
User ID: 974d13ad-05b9-4f3c-9b0f-d4cf4cad445b, User Vector: [31, 1, 5, 0, 2, 0, 1, 0]
User ID: dc632bfa-45ec-44fb-83fe-a27767fb2ac7, User Vector: [45, 1, 4, 3, 1, 0, 0, 3]
All User Vectors: [[20, 0, 0, 0, 0, 0, 0, 0], [32, 0, 0, 0, 0, 1, 0, 0], [26, 0, 0, 0, 0, 3, 2, 0], [21, 0, 0, 1, 0, 